In [2]:
#print schedule
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
#statement="SELECT count(*) FROM {} ORDER BY dow ASC, time ASC".format(table)
statement="SELECT * FROM {} ORDER BY dow ASC, time ASC".format(table)
#statement="SELECT * FROM {} WHERE dow=2 ORDER BY dow ASC, time ASC".format(table)
cursor=con.execute(statement)
rows = cursor.fetchall()
for row in rows:
    print(row)    
con.close()

(0, '00:00:01', 'shuffle_exclude/Fleetwood_Mac_-_The_Chain.mp3')
(0, '00:07:30', 'shuffle_exclude/1._The_Hand_That_Rocks_The_Cradle_Demo_-_The_Smiths.mp3')
(0, '00:15:00', 'shuffle_exclude/wilco-moremoon.mp3')
(0, '00:22:30', 'shuffle_exclude/Whiskeytown_-_Losering.mp3')
(0, '00:30:00', 'shuffle_exclude/cat_power-Love&Communication.mp3')
(0, '00:45:00', 'shuffle_exclude/Radiohead_-_The_Gloaming_-_Live_From_The_Basement_HD.mp3')
(0, '01:00:00', 'shuffle_exclude/throwing_muses-Golden_Thing.mp3')
(0, '01:30:00', 'shuffle_exclude/TheCure-TheLastDayofSummer.mp3')
(0, '02:00:00', 'shuffle_exclude/Field_Mice-Sensitive.mp3')
(0, '02:30:00', 'shuffle_exclude/cat_power-EmptyShell.mp3')
(0, '03:00:00', 'shuffle_exclude/Tom_Verlaine-Spiritual.mp3')
(0, '04:00:00', 'shuffle_exclude/beck-Horrible_Fanfare_Landslide_Exoskeleton.mp3')
(0, '05:00:00', 'shuffle_exclude/throwing_muses-Dylan.mp3')
(0, '06:00:00', 'shuffle_exclude/Wilco-OneSundayMorning.mp3')
(0, '06:30:00', 'shuffle_exclude/aha-TakeOnMe-Li

In [6]:
#add song
import sqlite3
from st_database import *
con = sqlite3.connect('soundtrack.db')
table='song_schedule'

if __name__=='__main__':
    days_of_week=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
    print_pick(days_of_week)
    dow=int_pick("Which day of week?: ",len(days_of_week))
    hour=int_pick("What hour of day?",24)
    hour=int(hour)
    if (hour < 10):
        hour='0'+str(hour)
    else:
        hour=str(hour)
    time_slots=['00:00','07:30','15:00','22:30','30:00','37:30','45:00','52:30']
    print_pick(time_slots)
    min_sec=int_pick("Which minute and second?: ",len(time_slots))
    min_sec=time_slots[int(min_sec)]
    song=input("What is the name of the song?: ")
    mil_time='{}:{}'.format(hour,min_sec)
    print("The data you entered looks like this: {},\'{}\',\'{}\'".format(dow,mil_time,song))
    if yes_or_no("Are you sure?"):
        try:
            statement ='INSERT INTO {} (name,dow,time) VALUES (\"{}\",{},\"{}\")'.format(table,song,dow,mil_time)
            con.execute(statement)
            con.commit()
            con.close()
            print("Successfully entered into database.\n{}".format(statement))
        except sqlite3.Error as e:
            statement="SELECT * from {} WHERE dow='{}' AND time='{}'".format(table,dow,mil_time)
            cursor=con.execute(statement)
            rows = cursor.fetchall()
            print("Time Slot taken by: \n{}\n{}".format(rows,e))
            con.close()
    else:
        print("Nothing entered into database.")

0->Sunday  1->Monday  2->Tuesday  3->Wednesday  4->Thursday  5->Friday  6->Saturday 
Which day of week?:  (0-6): 2
What hour of day? (0-23): 3
0->00:00  1->07:30  2->15:00  3->22:30  4->30:00  5->37:30  6->45:00  7->52:30 
Which minute and second?:  (0-7): 4
What is the name of the song?: test3
The data you entered looks like this: 2,'03:30:00','test3'
Are you sure? (y/n): n
Nothing entered into database.


In [ ]:
#find shuffle_exclude songs not on schedule yet
import sqlite3
import operator
import glob
pathMusic='/home/pi/Music/shuffle_exclude/'
filename_segment=pathMusic.count("/")
music_source="*.mp3"
music_list=glob.glob(pathMusic+music_source)
for song in music_list:
    song=song.split("/")
    #print(song[filename_segment])
    song="shuffle_exclude/"+str(song[filename_segment])
    #check db for count of previous plays
    con = sqlite3.connect('/home/pi/.jupyter/soundtrack/soundtrack.db')
    table='song_schedule'
    statement ='SELECT * FROM {} WHERE name="{}" LIMIT 0,1'.format(table,song)
    cursor=con.execute(statement)
    rows = cursor.fetchall()
    #print(len(rows))
    if len(rows) ==0:
        print(song)
        #print(rows)       
con.close()        

In [ ]:
#add song(old)
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
song,dow,tm='shuffle_exclude/Midlake-Young Bride.mp3',0,'11:52:30'
statement ='INSERT INTO {} (name,dow,time) VALUES (\"{}\",{},\"{}\")'.format(table,song,dow,tm)
con.execute(statement)
con.commit()
con.close()

In [3]:
#switch 2 songs
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
song1="10_000_Maniacs_-_These_Are_Days.mp3"
song2="PostalService-Track05.mp3"
day,time=[],[]
for song in song1,song2:
    statement="SELECT * from {} WHERE name='{}'".format(table,song)
    cursor=con.execute(statement)
    rows = cursor.fetchall()
    for row in rows:
        day.append(row[0])
        time.append(row[1])
statement1 ='UPDATE {} SET name="{}" WHERE dow={} AND time="{}"'.format(table,song2,day[0],time[0])
con.execute(statement1)
print(statement1)
statement2 ='UPDATE {} SET name="{}" WHERE dow={} AND time="{}"'.format(table,song1,day[1],time[1])
print(statement2)
con.execute(statement2)
con.commit()
con.close()

UPDATE song_schedule SET name="PostalService-Track05.mp3" WHERE dow=2 AND time="14:37:30"
UPDATE song_schedule SET name="10_000_Maniacs_-_These_Are_Days.mp3" WHERE dow=2 AND time="16:00:00"


In [4]:
#remove song
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
dow,tm=2,'14:37:30'
statement ='DELETE FROM {} WHERE dow={} AND time="{}"'.format(table,dow,tm)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#update song
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
statement ='UPDATE {} SET name="The GooGooDolls-BlackBalloon-Live_Acoustic.mp3" WHERE dow=2 AND time="10:52:30"'.format(table)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#analytics
import matplotlib.pyplot as plt
import pickle
filename='music_dict.p'
pickle=pickle.load(open(filename,"rb"))
music_dict=pickle
song_durations=[]
for song in music_dict:
    b=song[1][2]
    #print(b)
    song_durations.append(b) 
plt.plot(sorted(song_durations))
plt.show()

In [ ]:
#Tool
import pickle
from soundtrack import *
import sqlite3

filename='music_dict.p'
pickle=pickle.load(open(filename,"rb"))
music_dict=pickle
con=sqlite3.connect('soundtrack.db')
#print(music_dict)
for song in music_dict:
    b=song[1][2]
    print(b) 
#    statement='SELECT * FROM shuffle_count WHERE name="{}"'.format(song[0])
    statement='UPDATE shuffle_count SET duration_sec={} WHERE name="{}"'.format(b,song[0])  
    con.execute(statement)                                                                                                               
    #    rows = cursor.fetchall()                                                                                                                    
#    for row in rows:                                                                                                                            
#        print(row)                                                                                                                            
con.commit()
con.close() 

In [ ]:
#Tool2
import sqlite3

con=sqlite3.connect('soundtrack.db')
statement='SELECT * FROM shuffle_count' 
cursor=con.execute(statement)                                                                                                               
rows = cursor.fetchall()                                                                                                                    
for row in rows:                                                                                                                                
    print(row)                                                                                                                            
con.close() 

In [ ]:
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule2'
#query='DROP TABLE {}'.format(table)
query = 'CREATE TABLE {} (dow INTEGER NOT NULL,time TEXT NOT NULL,name TEXT NOT NULL, PRIMARY KEY (dow,time))'.format(table)
con.execute(query)
con.commit()
con.close()

In [ ]:
import sqlite3
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
#query='DROP TABLE {}'.format(table)
query = 'CREATE TABLE {} (dow INTEGER NOT NULL,time TEXT NOT NULL,name TEXT NOT NULL)'.format(table)
con.execute(query)
con.commit()
con.close()

In [ ]:
#to update table
import sqlite3
con = sqlite3.connect('soundtrack.db')
t='song_schedule'
t2='song_schedule2'
statement='INSERT INTO {} SELECT * FROM {}'.format(t2,t)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#purge old songs from shuffle_count
from soundtrack import *
mode=getShuffleCountMode()
#print(mode)
import sqlite3

con = sqlite3.connect('soundtrack.db')
table='shuffle_count'
statement ="DELETE FROM {} WHERE cnt<'{}'".format(table,mode-4)
con.execute(statement)
con.commit()
con.close()

In [ ]:
#check schedule
#todo: 20190530 find the three closest scheduled songs to now, replace one with a new song, 
#and move the original to nearby time slot
import sqlite3
from datetime import datetime, timedelta
con = sqlite3.connect('soundtrack.db')
table='song_schedule'
current_dow=datetime.now().strftime('%w')
current_time=datetime.now().strftime('%H:%M:%S')
end_tm=datetime.now() + timedelta(0,float(3000))
end_tm=end_tm.strftime('%H:%M:%S')
beg_tm=datetime.now() + timedelta(0,float(-3000))
beg_tm=beg_tm.strftime('%H:%M:%S')
print(current_dow,current_time)
print("Ends @ {}".format(end_tm))
statement="SELECT * FROM {} WHERE dow={} ORDER BY dow ASC, time ASC".format(table,current_dow)
cursor=con.execute(statement)
rows = cursor.fetchall()
for row in rows:
    if str(beg_tm) < row[1] < str(end_tm):
        print(row[1:3])    
con.close()

In [ ]:
import sqlite3 
from datetime import datetime

#test and implement this function below when you get a chance 20190709
def getShuffleCountMode():
    
    con = sqlite3.connect('soundtrack.db')
    table='shuffle_count'
    statement ='SELECT cnt,COUNT(name) shuffle_cnt FROM {} GROUP BY cnt ORDER BY shuffle_cnt ASC'.format(table)
    cursor=con.execute(statement)
    rows = cursor.fetchall()
    for row in rows:
        mode=row[0]
    con.close()
    return mode

mode=getShuffleCountMode()
print(mode)

In [ ]:
import sqlite3

con = sqlite3.connect('soundtrack.db')
table='shuffle_count'
statement ='SELECT * FROM {} ORDER BY cnt ASC'.format(table)
cursor=con.execute(statement)
rows = cursor.fetchall()
for row in rows:
    print(row)
con.close()